In [ ]:
!pip install datasets pillow transformers opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip uninstall torch torchvision torchaudio


Found existing installation: torch 1.13.1+cu117
Uninstalling torch-1.13.1+cu117:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-1.13.1+cu117.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-1.13.1+cu117
Found existing installation: torchvision 0.14.1+cu117
Uninstalling torchvision-0.14.1+cu117:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchvision-0.14.1+cu117.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libcudart.782fcab0.so.11.0
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libnvjpeg.9b083ad7.so.11
    /usr/local/lib/python3.10/dist-package

In [ ]:
!pip install torchvision torchaudio


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second) and total frame count
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Function to generate caption from a frame
def generate_description(frame):
    inputs = processor(images=frame, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

frame_rate = 1  # Extract 1 frame per second
frame_count = 0
scene_interval = 10  # Describe the scene every 10 seconds
frame_list = []

print("Starting frame extraction...")

# Read the video frames
while cap.isOpened() and (frame_count / fps) <= video_duration:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    if frame_count % (frame_rate * scene_interval * fps) == 0 and frame_list:
        # After every 10 seconds, generate description for the scene
        print(f"Generating description at {frame_count // fps} seconds...")
        description = generate_description(frame_list[-1])
        print(f"Scene Description at {frame_count // fps} seconds: {description}")
        frame_list = []  # Reset for the next 10 seconds

    if frame_count % (frame_rate * fps) == 0:
        # Append the frame every second
        frame_list.append(frame)

    frame_count += 1

cap.release()
print("Video processing completed.")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...
Using device: cpu
Starting frame extraction...
Generating description at 10.0 seconds...
Scene Description at 10.0 seconds: a car driving down a road next to trees
Generating description at 20.0 seconds...
Scene Description at 20.0 seconds: a video camera is shown in the middle of a wall
Generating description at 30.0 seconds...
Scene Description at 30.0 seconds: a car driving down a road next to a building
Generating description at 40.0 seconds...
Scene Description at 40.0 seconds: a car is driving down a road with a tire
Generating description at 50.0 seconds...
Scene Description at 50.0 seconds: a black car driving down a road
Generating description at 60.0 seconds...
Scene Description at 60.0 seconds: sko logo
End of video or error reading frame.
Video processing completed.


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second) and total frame count
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Function to generate caption from a frame
def generate_description(frame):
    inputs = processor(images=frame, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

frame_rate = 1  # Extract 1 frame per second
frame_count = 0
scene_interval = 10  # Describe the scene every 10 seconds
frame_list = []

print("Starting frame extraction...")

# Function to average frames or select the middle frame
def process_frames(frames):
    # Option 1: Average the frames (if averaging makes sense for your use case)
    avg_frame = np.mean(frames, axis=0).astype(np.uint8)

    # Option 2: Select the middle frame
    # middle_frame = frames[len(frames) // 2]

    return avg_frame

# Read the video frames
while cap.isOpened() and (frame_count / fps) <= video_duration:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    if frame_count % (frame_rate * scene_interval * fps) == 0 and frame_list:
        # Process the collected frames (average or select a frame)
        combined_frame = process_frames(frame_list)

        # Generate description for the combined frame
        print(f"Generating description at {frame_count // fps} seconds...")
        description = generate_description(combined_frame)
        print(f"Scene Description at {frame_count // fps} seconds: {description}")

        # Reset frame list for the next 10-second interval
        frame_list = []

    if frame_count % (frame_rate * fps) == 0:
        # Append the frame every second
        frame_list.append(frame)

    frame_count += 1

cap.release()
print("Video processing completed.")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...
Using device: cpu
Starting frame extraction...
Generating description at 10.0 seconds...
Scene Description at 10.0 seconds: a car driving down a road with trees in the background
Generating description at 20.0 seconds...
Scene Description at 20.0 seconds: a bus is parked in front of a building
Generating description at 30.0 seconds...
Scene Description at 30.0 seconds: a car dashboard with a digital display
Generating description at 40.0 seconds...
Scene Description at 40.0 seconds: a painting of a city with a lot of buildings
Generating description at 50.0 seconds...
Scene Description at 50.0 seconds: a car driving down a road with a speedometer
Generating description at 60.0 seconds...
Scene Description at 60.0 seconds: a sign that says the name of the company
End of video or error reading frame.
Video processing completed.


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second) and total frame count
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Function to generate caption from an image
def generate_description(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# Function to concatenate frames into a grid
def concatenate_frames(frames):
    # Resize all frames to a fixed size, say 256x256 for simplicity
    resized_frames = [cv2.resize(frame, (256, 256)) for frame in frames]

    # Concatenate frames into a single image grid (2 rows, 5 columns for 10 frames)
    grid = np.vstack([np.hstack(resized_frames[:5]), np.hstack(resized_frames[5:])])

    return grid

frame_rate = 1  # Extract 1 frame per second
frame_count = 0
scene_interval = 10  # Describe the scene every 10 seconds
frame_list = []

print("Starting frame extraction...")

# Read the video frames
while cap.isOpened() and (frame_count / fps) <= video_duration:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Append frame every second
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    # After every 10 seconds, concatenate and describe the scene
    if frame_count % (scene_interval * int(fps)) == 0 and frame_list:
        print(f"Generating description at {frame_count // fps} seconds...")

        # Concatenate frames into a single image
        concatenated_image = concatenate_frames(frame_list)

        # Generate description for the combined image
        description = generate_description(concatenated_image)
        print(f"Scene Description at {frame_count // fps} seconds: {description}")

        # Reset frame list for the next 10-second interval
        frame_list = []

    frame_count += 1

cap.release()
print("Video processing completed.")


Error: Could not open video.


ZeroDivisionError: float division by zero

In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Function to generate caption from an image
def generate_description(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=50)

    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# Function to concatenate frames into a grid
def concatenate_frames(frames, grid_rows, grid_cols):
    # Resize all frames to a fixed size (e.g., 256x256)
    resized_frames = [cv2.resize(frame, (256, 256)) for frame in frames]

    # Build the grid of frames
    rows = []
    for i in range(0, len(resized_frames), grid_cols):
        row_frames = resized_frames[i:i + grid_cols]
        if len(row_frames) < grid_cols:
            # Fill in with blank frames if row is not complete
            row_frames += [np.zeros_like(row_frames[0]) for _ in range(grid_cols - len(row_frames))]

        row = np.hstack(row_frames)
        rows.append(row)

    # Concatenate all rows into a final grid image
    grid_image = np.vstack(rows)

    return grid_image

# Extract 1 frame per second from the video
frame_list = []
frame_count = 0

print("Extracting 1 frame per second from the video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second (using FPS to calculate frame intervals)
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(frame_list)} frames from the video.")

# Concatenate all frames into a single grid
grid_rows = 10  # Number of rows in the grid
grid_cols = 10  # Number of columns in the grid (adjust based on total frames)

if len(frame_list) > grid_rows * grid_cols:
    # Limit the number of frames to fit into the grid
    frame_list = frame_list[:grid_rows * grid_cols]

# Create the grid image
grid_image = concatenate_frames(frame_list, grid_rows, grid_cols)

# Generate a description for the whole video using the grid of frames
print("Generating description for the entire video...")
description = generate_description(grid_image)
print(f"Video Description: {description}")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...
Using device: cuda
Extracting 1 frame per second from the video...
End of video or error reading frame.
Extracted 61 frames from the video.
Generating description for the entire video...
Video Description: a barcode with a black background


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Function to generate caption from an image
def generate_description(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=50)  # Increase max_new_tokens if you want longer descriptions
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# Extract 1 frame per second from the video
frame_list = []
frame_count = 0

print("Extracting 1 frame per second from the video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second (using FPS to calculate frame intervals)
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(frame_list)} frames from the video.")

# Generate descriptions for all frames and combine them
final_description = ""

print("Generating descriptions for each frame...")
for i, frame in enumerate(frame_list):
    description = generate_description(frame)
    print(f"Frame {i+1} Description: {description}")
    final_description += f"{description}. "

print(f"\nFinal Video Description: {final_description}")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...
Using device: cuda
Extracting 1 frame per second from the video...
End of video or error reading frame.
Extracted 61 frames from the video.
Generating descriptions for each frame...
Frame 1 Description: a black background with a white and red light
Frame 2 Description: a close up of a yellow car with a white emblem
Frame 3 Description: a close up of a yellow car with a black and white emblem
Frame 4 Description: a yellow car driving down a road next to a forest
Frame 5 Description: a yellow car driving down a road next to a forest
Frame 6 Description: a car driving down a road with trees in the background
Frame 7 Description: a yellow car driving down a road in the rain
Frame 8 Description: a yellow car driving down a road next to a forest
Frame 9 Description: a yellow car driving down a road in the rain
Frame 10 Description: a car driving down a road next to trees
Frame 11 Descript

In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, BartForConditionalGeneration, BartTokenizer

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Initialize BART model for summarization
print("Loading BART summarization model...")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)
bart_model.to(device)

# Function to generate caption from an image using BLIP
def generate_description(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=50)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# Function to summarize text using BART
def summarize_text(text):
    inputs = bart_tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)
    summary_ids = bart_model.generate(
        inputs['input_ids'],
        max_length=150,
        min_length=60,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Extract 1 frame per second from the video
frame_list = []
frame_count = 0

print("Extracting 1 frame per second from the video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second (using FPS to calculate frame intervals)
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(frame_list)} frames from the video.")

# Generate descriptions for all frames and combine them
frame_descriptions = []

print("Generating descriptions for each frame...")
for i, frame in enumerate(frame_list):
    description = generate_description(frame)
    print(f"Frame {i+1} Description: {description}")
    frame_descriptions.append(description)

# Concatenate all frame descriptions into a single text
combined_descriptions = " ".join(frame_descriptions)

# Summarize the combined descriptions using BART
print("\nSummarizing video description using BART...")
final_summary = summarize_text(combined_descriptions)

print(f"\nFinal Video Summary: {final_summary}")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...
Loading BART summarization model...
Using device: cuda
Extracting 1 frame per second from the video...
End of video or error reading frame.
Extracted 61 frames from the video.
Generating descriptions for each frame...
Frame 1 Description: a black background with a white and red light
Frame 2 Description: a close up of a yellow car with a white emblem
Frame 3 Description: a close up of a yellow car with a black and white emblem
Frame 4 Description: a yellow car driving down a road next to a forest
Frame 5 Description: a yellow car driving down a road next to a forest
Frame 6 Description: a car driving down a road with trees in the background
Frame 7 Description: a yellow car driving down a road in the rain
Frame 8 Description: a yellow car driving down a road next to a forest
Frame 9 Description: a yellow car driving down a road in the rain
Frame 10 Description: a car driving down a 

In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, BartForConditionalGeneration, BartTokenizer

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Initialize BART model for summarization
print("Loading BART summarization model...")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)
bart_model.to(device)

# Function to generate caption from an image using BLIP
def summarize_text(text):
    # Tokenize the input text with padding and truncation
    inputs = bart_tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding=True
    ).to(device)

    # Generate summary with varied parameters
    summary_ids = bart_model.generate(
        inputs['input_ids'],
        max_length=400,  # Increased max_length for a potentially more detailed summary
        min_length=100,  # Increased min_length to ensure summary is comprehensive
        length_penalty=1.5,
        top_p=0.1,# Adjusted length_penalty for more fluent text
        do_sample=True,
        repetition_penalty=10.0,
        num_beams=6,  # Increased number of beams for better summary quality
        early_stopping=True
    )

    # Decode the generated summary
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Extract 1 frame per second from the video
frame_list = []
frame_count = 0

print("Extracting 1 frame per second from the video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second (using FPS to calculate frame intervals)
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(frame_list)} frames from the video.")

# Generate descriptions for all frames and combine them
frame_descriptions = []

print("Generating descriptions for each frame...")
for i, frame in enumerate(frame_list):
    description = generate_description(frame)
    print(f"Frame {i+1} Description: {description}")
    frame_descriptions.append(description)

# Concatenate all frame descriptions into a single text
combined_descriptions = " ".join(frame_descriptions)

# Summarize the combined descriptions using BART
print("\nSummarizing video description using BART...")
final_summary = summarize_text(combined_descriptions)

print(f"\nFinal Video Summary: {final_summary}")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading BART summarization model...
Using device: cuda
Extracting 1 frame per second from the video...
End of video or error reading frame.
Extracted 61 frames from the video.
Generating descriptions for each frame...
Frame 1 Description: a black background with a white and red light
Frame 2 Description: a close up of a yellow car with a white emblem
Frame 3 Description: a close up of a yellow car with a black and white emblem
Frame 4 Description: a yellow car driving down a road next to a forest
Frame 5 Description: a yellow car driving down a road next to a forest
Frame 6 Description: a car driving down a road with trees in the background
Frame 7 Description: a yellow car driving down a road in the rain
Frame 8 Description: a yellow car driving down a road next to a forest
Frame 9 Description: a yellow car driving down a road in the rain
Frame 10 Description: a car driving down a road next to trees
Frame 11 Description: hyundai's new car is a gold car
Frame 12 Description: a yellow c

In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, BartForConditionalGeneration, BartTokenizer

# Load the video and set up the model for caption generation
video_path = "/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1) (1).mp4"
cap = cv2.VideoCapture(video_path)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the video's FPS (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

print(f"Video loaded successfully. FPS: {fps}, Total Frames: {total_frames}, Duration: {video_duration:.2f} seconds.")

# Initialize BLIP model for visual description
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Initialize BART model for summarization
print("Loading BART summarization model...")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)
bart_model.to(device)

# Function to generate caption from an image using BLIP
def generate_description(frame):
    # Preprocess the image
    inputs = processor(images=frame, return_tensors="pt").to(device)

    # Generate caption
    caption_ids = model.generate(**inputs)
    caption = processor.decode(caption_ids[0], skip_special_tokens=True)
    return caption

# Function to summarize text using BART
def summarize_text(text):
    # Tokenize the input text with padding and truncation
    inputs = bart_tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding=True
    ).to(device)

    # Generate summary with varied parameters
    summary_ids = bart_model.generate(
        inputs['input_ids'],
        max_length=400,  # Increased max_length for a potentially more detailed summary
        min_length=100,  # Increased min_length to ensure summary is comprehensive
        length_penalty=1.5,
        top_p=0.1, # Adjusted length_penalty for more fluent text
        do_sample=True,
        repetition_penalty=10.0,
        num_beams=6,  # Increased number of beams for better summary quality
        early_stopping=True
    )

    # Decode the generated summary
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Extract 1 frame per second from the video
frame_list = []
frame_count = 0

print("Extracting 1 frame per second from the video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second (using FPS to calculate frame intervals)
    if frame_count % int(fps) == 0:
        frame_list.append(frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(frame_list)} frames from the video.")

# Load the transcribed text with timestamps
transcript_file = "/content/transcription_with_timestamps.txt"
transcription_data = []

# Parse the text file
with open(transcript_file, "r") as f:
    for line in f:
        parts = line.strip().split(":")
        if len(parts) == 2:
            time_range, text = parts
            start_time, end_time = map(int, time_range.split("-"))
            transcription_data.append({"start": start_time, "end": end_time, "text": text.strip()})

# Combine visual descriptions and transcriptions based on timestamps
frame_descriptions = []

print("Generating descriptions for each frame and combining with transcription text...")

for i, frame in enumerate(frame_list):
    # Generate visual description for the frame
    description = generate_description(frame)

    # Get the transcription for the corresponding timestamp (assuming 1 frame per second)
    timestamp = i  # 1 frame per second, so use the index as the timestamp in seconds
    matching_transcription = ""

    # Find matching transcription based on timestamp
    for entry in transcription_data:
        start_time = entry['start']
        end_time = entry['end']
        if start_time <= timestamp <= end_time:
            matching_transcription = entry['text']
            break

    combined_description = f"Frame {i+1} Description: {description}. Transcript: {matching_transcription}"
    print(combined_description)
    frame_descriptions.append(combined_description)

# Concatenate all frame descriptions into a single text
combined_descriptions = " ".join(frame_descriptions)

# Summarize the combined descriptions using BART
print("\nSummarizing video description using BART...")
final_summary = summarize_text(combined_descriptions)

print(f"\nFinal Video Summary: {final_summary}")


Video loaded successfully. FPS: 25.0, Total Frames: 1511, Duration: 60.44 seconds.
Loading BLIP model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Using device: cuda
Extracting 1 frame per second from the video...
End of video or error reading frame.
Extracted 61 frames from the video.
Generating descriptions for each frame and combining with transcription text...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Frame 1 Description: a black background with a white and red light. Transcript: 
Frame 2 Description: a close up of a yellow car with a white emblem. Transcript: 
Frame 3 Description: a close up of a yellow car with a black and white emblem. Transcript: 
Frame 4 Description: a yellow car driving down a road next to a forest. Transcript: 
Frame 5 Description: a yellow car driving down a road next to a forest. Transcript: 
Frame 6 Description: a car driving down a road with trees in the background. Transcript: 
Frame 7 Description: a yellow car driving down a road in the rain. Transcript: 
Frame 8 Description: a yellow car driving down a road next to a forest. Transcript: 
Frame 9 Description: a yellow car driving down a road in the rain. Transcript: 
Frame 10 Description: a car driving down a road next to trees. Transcript: 
Frame 11 Description: hyundai's new car is a gold car. Transcript: 
Frame 12 Description: a yellow car driving down a road with trees in the background. Transcript:

In [ ]:
def parse_transcription(file_path):
    transcription_data = []

    # Open and read the transcription file
    with open(file_path, "r") as file:
        for line in file:
            # Strip any extra whitespace
            line = line.strip()

            # Find the position of the closing bracket and split the line
            if "]" in line:
                time_part, text = line.split("]", 1)
                time_part = time_part.replace("[", "").strip()  # Remove brackets and extra spaces
                text = text.strip()  # Clean the transcription text

                # Split the time_part into start and end times
                start_time, end_time = map(float, time_part.split(" - "))

                # Append the structured transcription data
                transcription_data.append({
                    "start_time": start_time,
                    "end_time": end_time,
                    "text": text
                })

    return transcription_data

# Example usage
file_path = "transcription_with_timestamps.txt"
parsed_transcriptions = parse_transcription(file_path)
for entry in parsed_transcriptions:
    print(entry)


{'start_time': 0.0, 'end_time': 4.2, 'text': 'There are many routes one can take in life.'}
{'start_time': 4.2, 'end_time': 7.64, 'text': 'Sometimes we take a beautiful'}
{'start_time': 7.64, 'end_time': 10.92, 'text': 'but measured journey.'}
{'start_time': 10.92, 'end_time': 14.92, 'text': 'But when life is full of choices...'}
{'start_time': 14.92, 'end_time': 21.92, 'text': 'sometimes you just let the heart take over.'}
{'start_time': 30.0, 'end_time': 45.92, 'text': 'When exhilaration drives you to aim higher.'}
{'start_time': 45.92, 'end_time': 51.36, 'text': 'When emotion overtakes all else.'}
{'start_time': 51.36, 'end_time': 52.88, 'text': 'ŠKODA SLAVIA 1.5 TSI'}


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Ensure the model uses GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
bart_model.to(device)

# Function to parse transcription text and prepare for BART summarization
def parse_and_summarize_transcription(file_path):
    transcription_data = []

    # Open and read the transcription file
    with open(file_path, "r") as file:
        full_transcription = ""
        for line in file:
            # Strip any extra whitespace
            line = line.strip()

            # Find the position of the closing bracket and split the line
            if "]" in line:
                time_part, text = line.split("]", 1)
                time_part = time_part.replace("[", "").strip()  # Remove brackets and extra spaces
                text = text.strip()  # Clean the transcription text

                # Split the time_part into start and end times
                start_time, end_time = map(float, time_part.split(" - "))

                # Append the structured transcription data
                transcription_data.append({
                    "start_time": start_time,
                    "end_time": end_time,
                    "text": text
                })

                # Combine all transcriptions for summarization
                full_transcription += " " + text

    # Generate a summary of the combined transcriptions using BART
    summary = summarize_text(full_transcription.strip())
    return summary

# Function to summarize text using BART
def summarize_text(text):
    # Tokenize the input text
    inputs = bart_tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)

    # Generate a more creative and generative summary with BART
    summary_ids = bart_model.generate(
        inputs['input_ids'],
        max_length=400,  # Adjust as needed
        min_length=100,  # Adjust as needed
        temperature=0.9,  # Increased temperature for more creativity
        top_p=0.95,  # Use nucleus sampling to get more diverse tokens
        top_k=50,  # Allows sampling from top 50 likely tokens
        do_sample=True,  # Enable sampling instead of greedy decoding
        repetition_penalty=10.0,  # Reduce repetition in the output
        num_beams=4,  # Fewer beams for more generative results
        early_stopping=True
    )

    # Decode the summary and return it
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# Example usage
file_path = "transcription_with_timestamps.txt"
final_summary = parse_and_summarize_transcription(file_path)

# Print the summary
print(f"Summary of the transcription:\n{final_summary}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Summary of the transcription:
There are many routes one can take in life. Sometimes we take a beautiful but measured journey. But when life is full of choices... sometimes you just let the heart take over. When exhilaration drives you to aim higher. When emotion overtakes all else. ŠKODA SLAVIA 1.5 TSI  - The World's Best Dressed Men and Women, by Cate Blanchett (Claudia Wozniacki), priced between £30 - £50 per episode on BBC1/2 or £40 – £60 for two episodes each. For more information visit: http://www.bbc.co.uk/worldsbestdoubledown/.
